In [ ]:
import cv2
import tkinter as tk
from tkinter import filedialog
from ultralytics import YOLO


FRAME_W, FRAME_H = 800, 600

ZONES = {
    "North": (200, 0, 400, 200),
    "South": (200, 400, 400, 600),
    "East":  (400, 200, 600, 400),
    "West":  (0, 200, 200, 400),
}


def load_model():
    return YOLO("yolov8n.pt")  

def bbox_center(bbox):
    x1, y1, x2, y2 = map(int, bbox)
    return (x1 + x2) // 2, (y1 + y2) // 2

def in_zone(point, zone):
    x, y = point
    zx1, zy1, zx2, zy2 = zone
    return zx1 <= x <= zx2 and zy1 <= y <= zy2

def detect_vehicles(model, frame):
    results = model(frame)
    detections = []
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            if model.names[cls] in ["car", "bus", "truck", "motorbike"]:
                detections.append({
                    "cls": model.names[cls],
                    "xyxy": box.xyxy[0].cpu().numpy()
                })
    return detections

def decide_signals(zone_counts):
    max_zone = max(zone_counts, key=zone_counts.get)
    signals = {}
    for zone, count in zone_counts.items():
        if count == 0:
            signals[zone] = "Red"
        elif zone == max_zone:
            signals[zone] = "Green"
        else:
            signals[zone] = "Red"
    return signals


def main():
    
    root = tk.Tk()
    root.withdraw()
    image_path = filedialog.askopenfilename(
        title="Select a Traffic Image",
        filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")]
    )
    if not image_path:
        print("❌ No file selected.")
        return

    frame = cv2.imread(image_path)
    if frame is None:
        print("❌ Error: Could not read image. Check the path.")
        return
    frame = cv2.resize(frame, (FRAME_W, FRAME_H))


    print("🔄 Loading YOLO model...")
    model = load_model()

    
    detections = detect_vehicles(model, frame)

    for d in detections:
        x1, y1, x2, y2 = map(int, d["xyxy"])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cx, cy = bbox_center(d["xyxy"])
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

    
    zone_counts = {}
    for name, zone in ZONES.items():
        count = 0
        for d in detections:
            cx, cy = bbox_center(d["xyxy"])
            if in_zone((cx, cy), zone):
                count += 1
        zone_counts[name] = count
        x1, y1, x2, y2 = zone
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

   
    signals = decide_signals(zone_counts)

    print("\n🚦 Adaptive Traffic Signal Decisions:")
    for zone, sig in signals.items():
        print(f"{zone}: {sig}")

   
    for name, zone in ZONES.items():
        x1, y1, _, _ = zone
        sig = signals.get(name, "Red")
        color = (0, 255, 0) if sig == "Green" else (0, 0, 255)
        cv2.putText(frame, f"{name}: {sig}", (x1, y1 + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

  
    cv2.imshow("Traffic Detection with Signals", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
